In [1]:
import numpy as np
import pandas as pd
import re
from datetime import datetime
pd.set_option('display.max_rows', None)

In [2]:
# read dealer list

dr = pd.read_excel('Transfer Order1_30_2023 2 24 00 AM.xlsx',header=4)

dr = dr[['Dealer','Region']]

dr = dr.sort_values('Region')

tela = dr[dr['Region']=='AP & Telangana'].reset_index(drop=True)
tela = tela.drop_duplicates().reset_index(drop=True)
tela.columns=tela.columns.str.lower()
tela.dealer = [i.lower() for i in tela.dealer]

In [4]:
tela

,dealer,region
0,the hindustan dental medical amp surgical corp...,AP & Telangana
1,shanti dental & medical co,AP & Telangana
2,gc ho-ap & telangana,AP & Telangana
3,universal dental suppliers,AP & Telangana
4,yashvi dental,AP & Telangana
5,deccan dental depot pvt. ltd.,AP & Telangana
6,sun dental distributors,AP & Telangana
7,crown dental depot,AP & Telangana
8,m/s. vmr dental suppliers private ltd,AP & Telangana
9,amrutha dental private ltd,AP & Telangana


In [5]:
path = r'C:\Users\stephenl\OneDrive - GC International AG\ML\whatsapp_india\\'

In [6]:
with open(path+'_chat.txt', encoding='utf-8') as f:
    c = f.readlines()

In [7]:
print(c)

['[2/2/23, 10:06:36 AM] Whatsapp Automation: \u200eMessages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them.\n', '[2/2/23, 10:06:36 AM] Whatsapp Automation: \u200eYou created group “Whatsapp Automation”\n', '[2/2/23, 10:25:14 AM] Madhu: \u200eMadhu joined using your invite\n', '[2/2/23, 10:25:22 AM] Madhu: 🙏🙏madhu\n', '[2/2/23, 10:26:03 AM] GC Dental: Ok. Now we are connected. I will create a message later and show you the result of the Automation\n', '[2/2/23, 1:00:23 PM] Madhu: 🙏Stephen\n', '[2/2/23, 2:14:16 PM] GC Dental: Raj mohan, apt dental,fuji ix capsule, 6, 18-Mar-23\n', '[3/2/23, 10:13:07 AM] GC Dental: K.n. neelima devi,apt dental, fuji ix capsule, 6, 18-Mar-23\n', '[3/2/23, 11:54:12 AM] GC Dental: K.n. neelima devi,amrutha dental, fuji ix capsule, 6, 18-Mar-23\n']


In [8]:
c[-1]

'[3/2/23, 11:54:12 AM] GC Dental: K.n. neelima devi,amrutha dental, fuji ix capsule, 6, 18-Mar-23\n'

## Confirm regex expression

In [9]:
match = '\d/\d{1,2}/\d{1,2},\s\d{1,2}:\d{1,2}:\d{1,2}\s?(AM|PM)'

In [10]:
match1 = '\[.*?\]'

In [11]:
f=[]
for i in range(len(c)):
    if re.search(match1,c[i]):
        f.append(c[i])
    else:
        continue

In [12]:
f

['[2/2/23, 10:06:36 AM] Whatsapp Automation: \u200eMessages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them.\n',
 '[2/2/23, 10:06:36 AM] Whatsapp Automation: \u200eYou created group “Whatsapp Automation”\n',
 '[2/2/23, 10:25:14 AM] Madhu: \u200eMadhu joined using your invite\n',
 '[2/2/23, 10:25:22 AM] Madhu: 🙏🙏madhu\n',
 '[2/2/23, 10:26:03 AM] GC Dental: Ok. Now we are connected. I will create a message later and show you the result of the Automation\n',
 '[2/2/23, 1:00:23 PM] Madhu: 🙏Stephen\n',
 '[2/2/23, 2:14:16 PM] GC Dental: Raj mohan, apt dental,fuji ix capsule, 6, 18-Mar-23\n',
 '[3/2/23, 10:13:07 AM] GC Dental: K.n. neelima devi,apt dental, fuji ix capsule, 6, 18-Mar-23\n',
 '[3/2/23, 11:54:12 AM] GC Dental: K.n. neelima devi,amrutha dental, fuji ix capsule, 6, 18-Mar-23\n']

In [13]:
def content(i):
    return re.split(match1,f[i])[1]

In [14]:
def timestamp(i):
    s = re.findall(match1,f[i])
    s1 = ','.join(s)
    s2 = s1.replace('[','').replace(']','')
    return s2.split(", ")[0]

In [15]:
def author(i):
    c = content(i)
    c1 = c.split(':')[0]
    return ' '.join(c1.split())

In [16]:
df = pd.DataFrame(columns=['date','author','content'])

In [17]:
df1=pd.DataFrame(columns=df.columns)

for i in range(len(f)):
    d = timestamp(i)
#     d = datetime.strptime(d, '%m/%d/%y')
    a = author(i)
    c = content(i)
    dic ={'date':d,'author':a, 'content':c}
    df = pd.DataFrame(dic,index=[0])
    df1=pd.concat([df1,df])
    df1.reset_index(drop=True, inplace=True)
    
# regex for date
datematch = r'^\d{1,2}/\d{1,2}/\d{1,2}$'
    
# remove non dates in date column
df3 = pd.DataFrame(columns=df1.columns)
df2 = pd.DataFrame(columns=df1.columns)
for idx, d in zip(df1.index,df1.date):
    if re.search(datematch,d):
        df2 = df1.loc[[idx]]
        df3 = pd.concat([df3,df2])
        
df4 = pd.DataFrame(columns=df1.columns)
df5 = pd.DataFrame(columns=df1.columns)
# remove content without ':'
for idx, c in zip(df3.index,df3.content):
    if re.search(r':',c):
        df4 = df3.loc[[idx]]
        df5 = pd.concat([df5,df4])

In [18]:
# remove whitespaces from date strings
df5.date = [' '.join(d.split()) for d in df5.date]

# convert to datetime object
df5.date = pd.to_datetime(df3.date,infer_datetime_format=True)

In [19]:
# remove text before colon

df5.content = [u.split(':')[1] for u in df5.content]

In [20]:
df5

,date,author,content
0,2023-02-02,Whatsapp Automation,‎Messages and calls are end-to-end encrypted....
1,2023-02-02,Whatsapp Automation,‎You created group “Whatsapp Automation”\n
2,2023-02-02,Madhu,‎Madhu joined using your invite\n
3,2023-02-02,Madhu,🙏🙏madhu\n
4,2023-02-02,GC Dental,Ok. Now we are connected. I will create a mes...
5,2023-02-02,Madhu,🙏Stephen\n
6,2023-02-02,GC Dental,"Raj mohan, apt dental,fuji ix capsule, 6, 18-..."
7,2023-03-02,GC Dental,"K.n. neelima devi,apt dental, fuji ix capsule..."
8,2023-03-02,GC Dental,"K.n. neelima devi,amrutha dental, fuji ix cap..."


In [35]:
df3 = df5[df5.date==df5.date.max()].reset_index(drop=True)

In [36]:
df3 = df3.iloc[[-1]].reset_index(drop=True)

In [37]:
df3

,date,author,content
0,2023-03-02,GC Dental,"K.n. neelima devi,amrutha dental, fuji ix cap..."


In [38]:
# # remove text before colun

# u ='H&D group: ‎Messages and calls are end-to-end.'
# u.split(':')[1]

In [39]:
df3.to_csv(path+'whatsappmsg.csv',index=False)

In [40]:
df3

,date,author,content
0,2023-03-02,GC Dental,"K.n. neelima devi,amrutha dental, fuji ix cap..."


In [41]:
# # 
# def extpdt(pdt):
#     if re.search(r'^fuji.+ix.+cap',pdt):
#         xx = 'Fuji IX GP Capsule'
#     elif re.search(r'^fuji.+ix.+extra',pdt):
#         xx = 'Fuji IX GP Extra'
#     return xx

# remove white spaces
def rs(a):
    return ' '.join(a.split())

In [47]:
# current date
n = datetime.now().strftime("%d-%b-%y")

# current time

In [48]:
s = df3.iloc[0,2]
s = s.lower()

In [49]:
# s = 'saleperson1: Aryyana dental lab,appolla impex,fuji ix  capsule,7,18-Apr-23'
# s = s.lower()

In [50]:
# transform message (clinic,dealer,product,qty,deliverydate)
m1 = s.split(':')[-1]
m1 = m1.split(',')


In [51]:
# define class to categorize message into (clinic,dealer,product,qty,deliverydate)
class gcindia():
    def __init__(self,clinic,dealer,product,qty,currentdate,deliverydate):
        self.clinic = clinic
        self.dealer = dealer
        self.product = product
        self.qty = qty
        self.currentdate = n
        self.deliverydate = deliverydate

In [52]:
# assign values to class s1
s1 = gcindia(rs(m1[0]),rs(m1[1]),rs(m1[2]),rs(m1[3]),n,rs(m1[4]))

In [53]:
s1.product, s1.currentdate

('fuji ix capsule', '28-Apr-23')

In [54]:
df = pd.DataFrame({"date": '1/27/23',"author":"salesperson1","clinic":s1.clinic,"dealer":s1.dealer,"product":s1.product,
                  "qty":s1.qty, "currentdate":s1.currentdate,"deliverydate":s1.deliverydate},index=[0])

In [55]:
df

,date,author,clinic,dealer,product,qty,currentdate,deliverydate
0,1/27/23,salesperson1,k.n. neelima devi,amrutha dental,fuji ix capsule,6,28-Apr-23,18-mar-23


In [56]:
clinic = df.loc[0,'clinic'].lower()
dealer = df.loc[0,'dealer'].lower()
product = df.loc[0,'product'].lower()
qty = df.loc[0,'qty'].lower()
deliverydate = df.loc[0,'deliverydate'].lower()

# clinic = 'k.n. neelima devi'
# clinic = clinic.lower()

# Read customer list & product list

In [57]:
# read customer list
df1 = pd.read_excel('gcindiacustomers.xlsx',header=4)
df1 = df1.dropna(subset=['Customer Name'])
df1 = df1[['Customer Name','Ship To Address','Class','Region']]
df1 = df1[df1['Region']=='AP & Telangana']
df1['Customer Name']= [i.lower() for i in df1['Customer Name']]
df1 = df1.reset_index(drop=True)

# read product list
pdt = pd.read_excel('gcindia_productlist.xlsx')
pdt.productnames = pdt.productnames.str.lower()

# Loop thru customer, product and dealer list

In [58]:
# extract the first word
def extract(k):
    return k.split(' ')[0]
def extract2(k):
    return k.split(' ')[0],k.split(' ')[1]
def extract3(k):
    return k.split(' ')[0],k.split(' ')[1],k.split(' ')[2]
def extract4(k):
    return k.split(' ')[0],k.split(' ')[1],k.split(' ')[2],k.split(' ')[3]
def extract5(k):
    return k.split(' ')[0],k.split(' ')[1],k.split(' ')[2],k.split(' ')[3],k.split(' ')[4]
def extract6(k):
    return k.split(' ')[0],k.split(' ')[1],k.split(' ')[2],k.split(' ')[3],k.split(' ')[4],k.split(' ')[5]
def extract7(k):
    return k.split(' ')[0],k.split(' ')[1],k.split(' ')[2],k.split(' ')[3],k.split(' ')[4],k.split(' ')[5],k.split(' ')[6]

def today():
    return datetime.now()

In [59]:
# f = extract('rohini n.')
# re.search(fr'{f}','raj mohan')

In [60]:
# for i in df1['Customer Name']:
#     f = extract2(i)
#     if re.search(fr'{f}','raj mohan'):
#         print(i)

In [61]:
# shorten the string

def shorten(k):
    add=[]
    k=k.replace('.','')
    length = len(k.split(' '))
    r = (k.split(' '))
    for s in range(length):
        if len(r[s]) < 3:
            continue
        else:
            add.append(r[s])

#     return ','.join(add)
    return add
    
def rembrac(j):
    return j.replace('(',' ').replace(')',' ')

In [62]:
# Loop thru customer list
df3 = pd.DataFrame(columns=['index','customerName','address','region','select_customer','product','select_product',
                            'dealer','select_dealer'])
h=0
g = clinic
for idx,n,a,r in zip(df1.index,df1['Customer Name'],df1['Ship To Address'],df1['Region']):
    nn = shorten(n)
#     length = len(nn.split(' '))
    length = len(nn)
    print(idx,nn,length)
    if length > 2:
#         f = extract3(nn)
        f = nn
        if re.search(fr'{f[0]}.+{f[1]}.+{f[2]}',g):
            df3.loc[h,'index']=idx
            df3.loc[h,'customerName']=n
            df3.loc[h,'address']=a
            df3.loc[h,'region']=r
            h=h+1
    elif length ==2:
#         f = extract2(nn)
        f = nn
        if re.search(fr'{f[0]}.+{f[1]}',g):
            df3.loc[h,'index']=idx
            df3.loc[h,'customerName']=n
            df3.loc[h,'address']=a
            df3.loc[h,'region']=r
            h=h+1
    elif length == 1:
#         f = extract(nn)
        f = ','.join(nn)
        if re.search(fr'{f}',g):
            df3.loc[h,'index']=idx
            df3.loc[h,'customerName']=n
            df3.loc[h,'address']=a
            df3.loc[h,'region']=r
            h=h+1

    df3=df3.reset_index(drop=True)
    df3.loc[h]=0

0 ['rohini'] 1
1 ['neelima', 'devi'] 2
2 ['ajay', 'kumar'] 2
3 ['bms', 'shankarlal'] 2
4 ['manmohan'] 1
5 ['murthy'] 1
6 ['sridhar', 'reddy'] 2
7 ['murli', 'krishna'] 2
8 ['faria', 'saba'] 2
9 ['priyenov'] 1
10 ['sarjeev', 'singh', 'yadav'] 3
11 ['kanil'] 1
12 ['vara', 'prasad'] 2
13 ['amruth', 'raj'] 2
14 ['deshmukh'] 1
15 ['kavita'] 1
16 ['niraj', 'agarwal'] 2
17 ['deepthi'] 1
18 ['rsreedhar'] 1
19 ['kavitha'] 1
20 ['dilip', 'kumar', 'reddy'] 3
21 ['dsatyanarayana'] 1
22 ['nmahipal'] 1
23 ['vamshi', 'krishna'] 2
24 ['kramesh'] 1
25 ['anupriya', 'jaitly'] 2
26 ['sudhaar'] 1
27 ['ksreenidhi', 'reddy'] 2
28 ['ktulasi'] 1
29 ['surender', 'reddy'] 2
30 ['nsurender', 'reddy'] 2
31 ['ymalleshwar'] 1
32 ['kshravan', 'kumar'] 2
33 ['rajesh', 'rayapuri'] 2
34 ['rajani', 'kanth', 'pulluri'] 3
35 ['mramesh', 'babu'] 2
36 ['msridhar'] 1
37 ['srikanth'] 1
38 ['satya', 'prakash'] 2
39 ['sanjay'] 1
40 ['srinivas'] 1
41 ['tiwary'] 1
42 ['amrithagandhi'] 1
43 ['jai', 'sekhar', 'reddy'] 3
44 ['mahesh',

435 ['raghava', 'rao'] 2
436 ['sreenatha', 'reddy'] 2
437 ['vijay', 'reddy'] 2
438 ['majid'] 1
439 ['chandrashekara'] 1
440 ['hari', 'priya'] 2
441 ['bramha', 'reddy'] 2
442 ['brahmaji', 'rao'] 2
443 ['kavita'] 1
444 ['bharathi'] 1
445 ['jagadish'] 1
446 ['rajani', 'kumar'] 2
447 ['ravikanth'] 1
448 ['sreelakshmi'] 1
449 ['suresh', 'kumar', 'kumar'] 3
450 ['sushendar', 'reddy'] 2
451 ['mamata', 'patibanda'] 2
452 ['susheel', 'kumar'] 2
453 ['sitadevig'] 1
454 ['srikanth', 'chowdary'] 2
455 ['ching', 'changhu'] 2
456 ['obul', 'reddy'] 2
457 ['kalyani'] 1
458 ['krishnam', 'raju'] 2
459 ['srinivas'] 1
460 ['muralikrishnach'] 1
461 ['neelima', 'chakradhark'] 2
462 ['sabana'] 1
463 ['ravi', 'shankarpl'] 2
464 ['rama', 'swamyp'] 2
465 ['sandeep'] 1
466 ['geethak'] 1
467 ['venkata', 'sudheerm'] 2
468 ['siva', 'nageswararaod'] 2
469 ['ajaydeva'] 1
470 ['radha', 'krishnaan'] 2
471 ['srikanthk'] 1
472 ['sunil', 'chandrat'] 2
473 ['krishnachaitanya'] 1
474 ['prabhu', 'sekhar'] 2
475 ['hemalatha']

883 ['pavan', 'baldava'] 2
884 ['drtjayasimha', 'reddy'] 2
885 ['tvanaja', 'reddy'] 2
886 ['ashmasha'] 1
887 ['grishma', 'goud'] 2
888 ['bhasker'] 1
889 ['rupasree'] 1
890 ['saritha'] 1
891 ['kranti', 'kiran'] 2
892 ['jagan', 'mohan'] 2
893 ['mohan', 'red'] 2
894 ['sadhana'] 1
895 ['vijay', 'kumar'] 2
896 ['rahul'] 1
897 ['schakravathi'] 1
898 ['ramakrishna'] 1
899 ['giridhar', 'kumar'] 2
900 ['anjali'] 1
901 ['raghav', 'rao'] 2
902 ['mahendra', 'azad'] 2
903 ['narayana'] 1
904 ['irfan'] 1
905 ['mohan'] 1
906 ['daljeet'] 1
907 ['naveen', 'reddy'] 2
908 ['sumadhur', 'madineni'] 2
909 ['harsha'] 1
910 ['lakshminarayana'] 1
911 ['zahera', 'banu'] 2
912 ['kravi', 'varma'] 2
913 ['goutham', 'kandula'] 2
914 ['srilatha'] 1
915 ['jaya', 'prakash'] 2
916 ['vvenkaiah'] 1
917 ['anageshwar', 'rao'] 2
918 ['gopi', 'kumar'] 2
919 ['sailaja'] 1
920 ['laxmann', 'rao'] 2
921 ['sukumar', 'reddy'] 2
922 ['radika'] 1
923 ['sirisha'] 1
924 ['usha'] 1
925 ['vijaya', 'lakshmi'] 2
926 ['bkrishna', 'reddy'] 2

1337 ['kalyan'] 1
1338 ['navya'] 1
1339 ['snalini'] 1
1340 ['chsvramani'] 1
1341 ['bsrinivas'] 1
1342 ['gedward', 'goldwin'] 2
1343 ['gsupriya', 'edward'] 2
1344 ['baruna', 'sree'] 2
1345 ['pvkrishnam', 'raju'] 2
1346 ['dsraju'] 1
1347 ['ksita', 'rama', 'rao'] 3
1348 ['kkavitha'] 1
1349 ['msobba', 'rajesh'] 2
1350 ['ulakshman', 'varma'] 2
1351 ['sreedhar', 'bitra'] 2
1352 ['mkrishna', 'kishore'] 2
1353 ['kraja', 'venkatesh', 'murthy'] 3
1354 ['mrsjyothi', 'murthy'] 2
1355 ['mbhanu', 'prasad'] 2
1356 ['srikrishna', 'chalasani'] 2
1357 ['nsatish', 'varma'] 2
1358 ['nsrilatha'] 1
1359 ['mrsjayasri', 'chalasani'] 2
1360 ['tsunil'] 1
1361 ['ujvala'] 1
1362 ['sandeep', 'reddy'] 2
1363 ['braghavi'] 1
1364 ['narra', 'vrsseshu', 'prasad'] 3
1365 ['gpratyusha'] 1
1366 ['ppanduranga', 'raju'] 2
1367 ['siva', 'nagendra', 'prasad'] 3
1368 ['ksrinivasa', 'rao'] 2
1369 ['kbala', 'krishna'] 2
1370 ['vvraghu', 'ram'] 2
1371 ['pradeep', 'kumar'] 2
1372 ['mallesh'] 1
1373 ['nishi', 'kumar'] 2
1374 ['sati

1661 ['suresh', 'gorantla'] 2
1662 ['vsrijan', 'babu'] 2
1663 ['ckvdurga', 'bhavani'] 2
1664 ['cvprasanna', 'kumar'] 2
1665 ['klavanya'] 1
1666 ['kvramani'] 1
1667 ['vvdprasad'] 1
1668 ['jkalayana', 'chakravathi'] 2
1669 ['bvamsi', 'pavani'] 2
1670 ['svvvedavathi'] 1
1671 ['mjsudhir'] 1
1672 ['smksirisha'] 1
1673 ['bvamsi', 'priya'] 2
1674 ['pravi', 'sekhar'] 2
1675 ['pvvsuryanarayana', 'raju'] 2
1676 ['kvvpratap', 'varma'] 2
1677 ['tammineni', 'diwakar'] 2
1678 ['bvvsrinivas'] 1
1679 ['brupa'] 1
1680 ['dsramesh', 'babu'] 2
1681 ['prem', 'sakkhari'] 2
1682 ['john'] 1
1683 ['sunitha'] 1
1684 ['kala'] 1
1685 ['gvamsi', 'krishna'] 2
1686 ['vvani'] 1
1687 ['kalyan', 'chakravarthy'] 2
1688 ['aravikumar'] 1
1689 ['pkrishnam', 'raju'] 2
1690 ['konchada', 'narendra', 'kumar'] 3
1691 ['phemalatha'] 1
1692 ['sarada', 'challa'] 2
1693 ['silla', 'satish', 'kumar'] 3
1694 ['mhari', 'krishna'] 2
1695 ['balaga', 'ramesh', 'naidu'] 3
1696 ['svsnmurthy'] 1
1697 ['mjaya', 'krishna'] 2
1698 ['aramesh'] 1

...

3831 ['drharshitha', 'pati'] 2
3832 ['drkshravan', 'kumar'] 2
3833 ['drmdabdul', 'azeem'] 2
3834 ['drsukhender', 'sirangi'] 2
3835 ['drksudhakar'] 1
3836 ['drgrajender'] 1
3837 ['drjammu', 'martin'] 2
3838 ['drvshyam', 'sunder', 'reddy'] 3
3839 ['drusha', 'jagannadula'] 2
3840 ['drktulasi'] 1
3841 ['drkavitha', 'reddy'] 2
3842 ['drvenu', 'yadav'] 2
3843 ['drsubash', 'gattu'] 2
3844 ['subhash'] 1
3845 ['swamy'] 1
3846 ['rajamahendra', 'katari'] 2
3847 ['raghu', 'chaitanya'] 2
3848 ['niveditha'] 1
3849 ['sajjan', 'reddy'] 2
3850 ['vijay', 'babu'] 2
3851 ['charan', 'kumar', 'mdp'] 3
3852 ['prakash', 'ulavalapudi'] 2
3853 ['rameshs'] 1
3854 ['lakshmi', 'kanthk'] 2
3855 ['srinivas', 'raoravipati'] 2
3856 ['mahendrab'] 1
3857 ['dental', 'depot'] 2
3858 ['vedavathi'] 1
3859 ['vara', 'prasad', 'vzm'] 3
3860 ['mallikarjun', 'rao'] 2
3861 ['sandeep', 'reddy'] 2
3862 ['pradeep', 'kumar', 'reddy'] 3
3863 ['anuradha'] 1
3864 ['geeta', 'kumari'] 2
3865 ['drsuma', 'liz', 'panicker'] 3
3866 ['drsrikha

4269 ['priya'] 1
4270 ['jai', 'shekhar', 'reddy'] 3
4271 ['chaitanya', 'singuru'] 2
4272 ['namballa', 'prudhviraj'] 2
4273 ['drfaisal', 'ansari'] 2
4274 ['drprama', 'devi'] 2
4275 ['drvishal', 'singh'] 2
4276 ['rajkumar'] 1
4277 ['anil', 'kumar', 'adikoppula', 'madinaguda'] 4
4278 ['yamini'] 1
4279 ['manoj', 'kumar', 'bobbili'] 3
4280 ['shiva', 'prasad'] 2
4281 ['santhosha'] 1
4282 ['jhansi'] 1
4283 ['gautham'] 1
4284 ['suman', 'raju'] 2
4285 ['shivpriya'] 1
4286 ['haranadh', 'reddy'] 2
4287 ['hariesh', 'reddy'] 2
4288 ['anu', 'preeta'] 2
4289 ['nidesh'] 1
4290 ['drramesh', 'babu'] 2
4291 ['drsuhas', 'tadepally'] 2
4292 ['drgvinod', 'kumar'] 2
4293 ['drpavan', 'baldeva'] 2
4294 ['drdipti', 'baldava'] 2
4295 ['drkvenkatesh'] 1
4296 ['drbsshilpa'] 1
4297 ['drsubbaiah', 'naidu'] 2
4298 ['dranurag', 'lahoti'] 2
4299 ['drvsai', 'sagar', 'reddy'] 3
4300 ['drsaqib'] 1
4301 ['drnagarani'] 1
4302 ['malik'] 1
4303 ['phhani', 'kumar'] 2
4304 ['sreedhar', 'dental'] 2
4305 ['teja', 'varma'] 2
4306 

4715 ['chaing'] 1
4716 ['ramesh', 'babu'] 2
4717 ['rachana'] 1
4718 ['praveena'] 1
4719 ['madhukar', 'neerudu'] 2
4720 ['vamshy', 'krishna'] 2
4721 ['raju', 'singam', 'shetty'] 3
4722 ['balaji', 'ketineni'] 2
4723 ['bharath', 'reddy'] 2
4724 ['jaya', 'prakash', 'reddy'] 3
4725 ['vasu', 'murthy'] 2
4726 ['farheem', 'fathima'] 2
4727 ['anusha'] 1
4728 ['venu', 'gopal'] 2
4729 ['rama', 'krishna', 'raju'] 3
4730 ['vinod', 'tru'] 2
4731 ['sivaji'] 1
4732 ['ravindra', 'reddy'] 2
4733 ['bharath', 'vishal', 'mantri'] 3
4734 ['santhosh', 'challa'] 2
4735 ['vijay'] 1
4736 ['praveen', 'kumar'] 2
4737 ['vivek', 'shastrula'] 2
4738 ['vinod', 'pawar'] 2
4739 ['sri', 'charan', 'rao'] 3
4740 ['sai', 'madhavi', 'nallamilla'] 3
4741 ['sri', 'sujan', 'suryadevara'] 3
4742 ['usha', 'rani'] 2
4743 ['lokesh'] 1
4744 ['prabhu'] 1
4745 ['mahesh'] 1
4746 ['srivani', 'reddy'] 2
4747 ['abhignini'] 1
4748 ['kusuma'] 1
4749 ['mekala', 'ravali'] 2
4750 ['aparna'] 1
4751 ['uma', 'kiran'] 2
4752 ['vikram', 'kumar', '

5037 ['ashok', 'kumar', 'kondapur'] 3
5038 ['suhas', 'natuur', 'kondapur'] 3
5039 ['padmaja', 'pithni'] 2
5040 ['sameera'] 1
5041 ['swetha', 'sunshine'] 2
5042 ['vijay', 'kumar'] 2
5043 ['srinivas', 'alla'] 2
5044 ['suvarna', 'lavanya'] 2
5045 ['veenavi'] 1
5046 ['dheeraj', 'bandar'] 2
5047 ['sandeep', 'nanda', 'kumar'] 3
5048 ['sindhura'] 1
5049 ['amarender'] 1
5050 ['deepesh'] 1
5051 ['venkat', 'ratna', 'nag'] 3
5052 ['abhilash', 'ravi'] 2
5053 ['drpratibha'] 1
5054 ['nagineni', 'sri', 'harsha'] 3
5055 ['satish', 'kumar', 'kotha'] 3
5056 ['bhavana', 'shuttle'] 2
5057 ['maheswar', 'reddy'] 2
5058 ['vivek', 'kandanoor'] 2
5059 ['preeti', 'badham'] 2
5060 ['preethi'] 1
5061 ['praj', 'kumar'] 2
5062 ['kraja', 'rao'] 2
5063 ['amala'] 1
5064 ['bhaskar', 'kinthada'] 2
5065 ['sirisha'] 1
5066 ['sravya'] 1
5067 ['viswa', 'teja'] 2
5068 ['krishna', 'teja', 'bvrm'] 3
5069 ['sony', 'dental', 'depo'] 3
5070 ['vinod', 'kmd'] 2
5071 ['kalyan', 'ksvk'] 2
5072 ['sri', 'ram'] 2
5073 ['hari', 'krishna'

5340 ['sudhir', 'surapaneni'] 2
5341 ['praveen'] 1
5342 ['pavan', 'kumar'] 2
5343 ['malleswari'] 1
5344 ['kiran', 'chowdhary'] 2
5345 ['sudhakar', 'raju'] 2
5346 ['drsri', 'laxmi'] 2
5347 ['dramadhavi'] 1
5348 ['dranne', 'gopinath'] 2
5349 ['drrohinin'] 1
5350 ['drsawan', 'kulkarni'] 2
5351 ['dranupamas'] 1
5352 ['drdwijender'] 1
5353 ['drvijay', 'chowdary'] 2
5354 ['drmayuri'] 1
5355 ['drmramanath', 'reddy'] 2
5356 ['paladugu', 'jithendra', 'babu'] 3
5357 ['padmaja', 'pithani'] 2
5358 ['kswetha'] 1
5359 ['sriram'] 1
5360 ['aarti', 'saxena'] 2
5361 ['nsandeep'] 1
5362 ['chanrasekhar'] 1
5363 ['pavan', 'gopal'] 2
5364 ['asridevi'] 1
5365 ['madhu', 'vasepalli'] 2
5366 ['abhilash'] 1
5367 ['venu', 'gopal', 'rao'] 3
5368 ['swapna'] 1
5369 ['ajay', 'krishna'] 2
5370 ['chandrashekhar', 'reddy'] 2
5371 ['drharikrishna'] 1
5372 ['drravi', 'gurram'] 2
5373 ['drkishore', 'kumar'] 2
5374 ['dramruthraj'] 1
5375 ['dr,phani', 'praneetha', 'boppana'] 3
5376 ['drveenavi'] 1
5377 ['drvaraprasad'] 1
537

In [63]:
clinic

'k.n. neelima devi'

In [64]:
df1[df1['Customer Name']==clinic]

,Customer Name,Ship To Address,Class,Region
1,k.n. neelima devi,NaN,C,AP & Telangana
1197,k.n. neelima devi,NaN,C,AP & Telangana
1546,k.n. neelima devi,NaN,C,AP & Telangana
1581,k.n. neelima devi,NaN,C,AP & Telangana


In [65]:
df3

,index,customerName,address,region,select_customer,product,select_product,dealer,select_dealer
0,1,k.n. neelima devi,NaN,AP & Telangana,0,0,0,0,0
1,259,neelima,"Andhra Pradesh, India-",AP & Telangana,0,0,0,0,0
2,1197,k.n. neelima devi,NaN,AP & Telangana,0,0,0,0,0
3,1546,k.n. neelima devi,NaN,AP & Telangana,0,0,0,0,0
4,1581,k.n. neelima devi,NaN,AP & Telangana,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0


In [66]:
# # Loop thru customer list
# df3 = pd.DataFrame(columns=['index','customerName','address','class','region','select_customer','product','select_product',
#                             'dealer','select_dealer'])
# h=0
# g = clinic
# for idx,n,a,r in zip(df1.index,df1['Customer Name'],df1['Ship To Address'],df1['Region']):
#     nn = shorten(n)
# #     length = len(nn.split(' '))
#     length = len(nn)
#     print(idx,nn,length)
#     if length > 2:
# #         f = extract3(nn)
#         f = nn
#         if re.search(fr'{f[0]}',g) or re.search(fr'{f[1]}',g) or re.search(fr'{f[2]}',g):
#             df3.loc[h,'index']=idx
#             df3.loc[h,'customerName']=n
#             df3.loc[h,'address']=a
#             df3.loc[h,'region']=r
#             h=h+1
#     elif length ==2:
# #         f = extract2(nn)
#         f = nn
#         if re.search(fr'{f[0]}',g) or re.search(fr'{f[1]}',g):
#             df3.loc[h,'index']=idx
#             df3.loc[h,'customerName']=n
#             df3.loc[h,'address']=a
#             df3.loc[h,'region']=r
#             h=h+1
#     elif length == 1:
# #         f = extract(nn)
#         f = ','.join(nn)
#         if re.search(fr'{f}',g):
#             df3.loc[h,'index']=idx
#             df3.loc[h,'customerName']=n
#             df3.loc[h,'address']=a
#             df3.loc[h,'region']=r
#             h=h+1

#     df3=df3.reset_index(drop=True)
#     df3.loc[h]=0
    

In [67]:
# # loop thru product list
# g= product
# h=0
# for n in pdt.productnames:
#     length = len(n.split(' '))
#     if length > 2:
#         f = extract3(n)
#         if re.search(fr'{f[0]}.+{f[1]}.+{f[2]}',g):
#             print(n)
#             df3.loc[h,'product']=n
#             print(1,n)
#             h+=1
            
            
#     elif length == 2:
#         f = extract2(n)
#         if re.search(fr'{f[0]}.+{f[1]}',g):
#             df3.loc[h,'product']=n
#             print(2,n)
#             h+=1
            
            
#     elif length == 1:
#         f = extract(n)
#         if re.search(fr'{f}',g):
#             df3.loc[h,'product']=n
#             print(3,n,f)
#             h+=1
            

In [68]:
pdt

,productnames
0,g-bond
1,solare x kit
2,other
3,fuji bond lc
4,stickresin
5,g aenial bond
6,g premio bond
7,equia forte fill
8,dentin conditioner
9,epitex


In [69]:
# n = rembrac('solare universal bond (twin pack)')
# f = extract5(n)
# if re.search(fr'{f[0]}',g) or re.search(fr'{f[1]}',g) or re.search(fr'{f[2]}',g)\
#         or re.search(fr'{f[3]}',g) or re.search(fr'{f[4]}',g):
#     print(True)

In [70]:
# n = 'fuji ix gp extra capsule'
# f = extract5(n)
# if re.search(fr'{f[0]}',g) or re.search(fr'{f[1]}',g) or re.search(fr'{f[2]}',g)\
# or re.search(fr'{f[3]}',g) or re.search(fr'{f[4]}',g):
#     print(True)

In [71]:
# loop thru product list
g= product
h=0

for n in pdt.productnames:
    n = rembrac(n)
    length = len(n.split(' '))
    
    if length ==7:
        f = extract7(n)
        if re.search(fr'{f[0]}',g) or re.search(fr'{f[1]}',g) or re.search(fr'{f[2]}',g)\
        and re.search(fr'{f[3]}',g) or re.search(fr'{f[4]}',g) or re.search(fr'{f[5]}',g) or re.search(fr'{f[6]}',g):
            df3.loc[h,'product']=n
            h+=1
    
    elif length ==6:
        f = extract6(n)
        if re.search(fr'{f[0]}',g) or re.search(fr'{f[1]}',g) or re.search(fr'{f[2]}',g)\
        or re.search(fr'{f[3]}',g) or re.search(fr'{f[4]}',g) or re.search(fr'{f[5]}',g):
            df3.loc[h,'product']=n
            h+=1
    
    elif length ==5:
        f = extract5(n)
        if re.search(fr'{f[0]}',g) or re.search(fr'{f[1]}',g) or re.search(fr'{f[2]}',g)\
        or re.search(fr'{f[3]}',g) or re.search(fr'{f[4]}',g):
            df3.loc[h,'product']=n
            h+=1
    
    elif length ==4:
        f = extract4(n)
        if re.search(fr'{f[0]}',g) or re.search(fr'{f[1]}',g) or re.search(fr'{f[2]}',g)\
        or re.search(fr'{f[3]}',g):
            df3.loc[h,'product']=n
            h+=1
    
    elif length ==3:
        f = extract3(n)
        if re.search(fr'{f[0]}',g) or re.search(fr'{f[1]}',g) or re.search(fr'{f[2]}',g):
            df3.loc[h,'product']=n
            h+=1
            
            
    elif length == 2:
        f = extract2(n)
        if re.search(fr'{f[0]}',g) or re.search(fr'{f[1]}',g):
            df3.loc[h,'product']=n
            h+=1
            
            
    elif length == 1:
        f = extract(n)
        if re.search(fr'{f}',g):
            df3.loc[h,'product']=n
            h+=1
            

In [72]:
df3

,index,customerName,address,region,select_customer,product,select_product,dealer,select_dealer
0,1.0,k.n. neelima devi,NaN,AP & Telangana,0.0,solare x kit,0.0,0.0,0.0
1,259.0,neelima,"Andhra Pradesh, India-",AP & Telangana,0.0,fuji bond lc,0.0,0.0,0.0
2,1197.0,k.n. neelima devi,NaN,AP & Telangana,0.0,everstick -c&b- 2 x 12 cm,0.0,0.0,0.0
3,1546.0,k.n. neelima devi,NaN,AP & Telangana,0.0,everstick -c&b- 1 x 8 cm,0.0,0.0,0.0
4,1581.0,k.n. neelima devi,NaN,AP & Telangana,0.0,stickrefix l,0.0,0.0,0.0
5,0.0,0,0,0,0.0,everstick -perio 2 x 12,0.0,0.0,0.0
6,NaN,NaN,NaN,NaN,NaN,everstick -perio 1 x 8,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,capsule applier,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,capsule mixer,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,fuji ix gp extra capsule,NaN,NaN,NaN


In [73]:
tela

,dealer,region
0,the hindustan dental medical amp surgical corp...,AP & Telangana
1,shanti dental & medical co,AP & Telangana
2,gc ho-ap & telangana,AP & Telangana
3,universal dental suppliers,AP & Telangana
4,yashvi dental,AP & Telangana
5,deccan dental depot pvt. ltd.,AP & Telangana
6,sun dental distributors,AP & Telangana
7,crown dental depot,AP & Telangana
8,m/s. vmr dental suppliers private ltd,AP & Telangana
9,amrutha dental private ltd,AP & Telangana


In [74]:
# # loop thru dealer list
# g= dealer
# h=0
# for n in tela.dealer:
#     n = rembrac(n)
#     length = len(n.split(' '))
#     if length > 6:
#         f = extract7(n)
#         if re.search(fr'{f[0]}.+{f[1]}.+{f[2]}.+{f[3]}.+{f[4]}.+{f[5]}.+{f[6]}',g):
#             df3.loc[h,'dealer']=n
#             print(1,n)
#             h+=1
#     elif length > 5 and length <=6:
#         f = extract6(n)
#         if re.search(fr'{f[0]}.+{f[1]}.+{f[2]}.+{f[3]}.+{f[4]}.+{f[5]}',g):
#             df3.loc[h,'dealer']=n
#             print(1,n)
#             h+=1
#     elif length > 4 and length <=5:
#         f = extract5(n)
#         if re.search(fr'{f[0]}.+{f[1]}.+{f[2]}.+{f[3]}.+{f[4]}',g):
#             df3.loc[h,'dealer']=n
#             print(1,n)
#             h+=1
#     elif length > 3 and length <=4:
#         f = extract4(n)
#         if re.search(fr'{f[0]}.+{f[1]}.+{f[2]}.+{f[3]}:
#             df3.loc[h,'dealer']=n
#             print(1,n)
#             h+=1
#     elif length > 2 and length <=3:
#         f = extract3(n)
#         if re.search(fr'{f[0]}.+{f[1]}.+{f[2]}:
#             df3.loc[h,'dealer']=n
#             print(1,n)
#             h+=1
#     elif length == 2:
#         f = extract2(n)
#         if re.search(fr'{f[0]}.+{f[1]}:
#             df3.loc[h,'dealer']=n
#             print(2,n)
#             h+=1
#     elif length == 1:
#         f = extract(n)
#         if re.search(fr'{f}',g):
#             df3.loc[h,'dealer']=n
#             print(3,n,f)
#             h+=1
            

In [75]:
tela

,dealer,region
0,the hindustan dental medical amp surgical corp...,AP & Telangana
1,shanti dental & medical co,AP & Telangana
2,gc ho-ap & telangana,AP & Telangana
3,universal dental suppliers,AP & Telangana
4,yashvi dental,AP & Telangana
5,deccan dental depot pvt. ltd.,AP & Telangana
6,sun dental distributors,AP & Telangana
7,crown dental depot,AP & Telangana
8,m/s. vmr dental suppliers private ltd,AP & Telangana
9,amrutha dental private ltd,AP & Telangana


In [76]:
g

'fuji ix capsule'

In [77]:
n = 'amrutha dental private ltd'
if re.search(fr'{f[0]}',g) or re.search(fr'{f[1]}',g) or re.search(fr'{f[2]}',g)\
        or re.search(fr'{f[3]}',g):
    print(True)

True


In [78]:
# loop thru dealer list
g= dealer
h=0
for n in tela.dealer:
    n = rembrac(n)
    length = len(n.split(' '))
    if length >6:
        f = extract7(n)
        if re.search(fr'{f[0]}',g) or re.search(fr'{f[1]}',g) or re.search(fr'{f[2]}',g)\
        and re.search(fr'{f[3]}',g) or re.search(fr'{f[4]}',g) or re.search(fr'{f[5]}',g) or re.search(fr'{f[6]}',g):
            df3.loc[h,'dealer']=n
            h+=1
    
    elif length ==6:
        f = extract6(n)
        if re.search(fr'{f[0]}',g) or re.search(fr'{f[1]}',g) or re.search(fr'{f[2]}',g)\
        or re.search(fr'{f[3]}',g) or re.search(fr'{f[4]}',g) or re.search(fr'{f[5]}',g):
            df3.loc[h,'dealer']=n
            h+=1
    
    elif length ==5:
        f = extract5(n)
        if re.search(fr'{f[0]}',g) or re.search(fr'{f[1]}',g) or re.search(fr'{f[2]}',g)\
        or re.search(fr'{f[3]}',g) or re.search(fr'{f[4]}',g):
            df3.loc[h,'dealer']=n
            h+=1
    
    elif length ==4:
        f = extract4(n)
        if re.search(fr'{f[0]}',g) or re.search(fr'{f[1]}',g) or re.search(fr'{f[2]}',g)\
        or re.search(fr'{f[3]}',g):
            df3.loc[h,'dealer']=n
            h+=1
    
    elif length ==3:
        f = extract3(n)
        if re.search(fr'{f[0]}',g) or re.search(fr'{f[1]}',g) or re.search(fr'{f[2]}',g):
            df3.loc[h,'dealer']=n
            h+=1
            
            
    elif length == 2:
        f = extract2(n)
        if re.search(fr'{f[0]}',g) or re.search(fr'{f[1]}',g):
            df3.loc[h,'dealer']=n
            h+=1
            
            
    elif length == 1:
        f = extract(n)
        if re.search(fr'{f}',g):
            df3.loc[h,'dealer']=n
            h+=1

In [79]:
df3

,index,customerName,address,region,select_customer,product,select_product,dealer,select_dealer
0,1.0,k.n. neelima devi,NaN,AP & Telangana,0.0,solare x kit,0.0,shanti dental & medical co,0.0
1,259.0,neelima,"Andhra Pradesh, India-",AP & Telangana,0.0,fuji bond lc,0.0,universal dental suppliers,0.0
2,1197.0,k.n. neelima devi,NaN,AP & Telangana,0.0,everstick -c&b- 2 x 12 cm,0.0,yashvi dental,0.0
3,1546.0,k.n. neelima devi,NaN,AP & Telangana,0.0,everstick -c&b- 1 x 8 cm,0.0,deccan dental depot pvt. ltd.,0.0
4,1581.0,k.n. neelima devi,NaN,AP & Telangana,0.0,stickrefix l,0.0,sun dental distributors,0.0
5,0.0,0,0,0,0.0,everstick -perio 2 x 12,0.0,crown dental depot,0.0
6,NaN,NaN,NaN,NaN,NaN,everstick -perio 1 x 8,NaN,m/s. vmr dental suppliers private ltd,NaN
7,NaN,NaN,NaN,NaN,NaN,capsule applier,NaN,amrutha dental private ltd,NaN
8,NaN,NaN,NaN,NaN,NaN,capsule mixer,NaN,the dental lab,NaN
9,NaN,NaN,NaN,NaN,NaN,fuji ix gp extra capsule,NaN,NaN,NaN


In [80]:
# path for google drive

path_tela = r'G:\My Drive\GC_India\\'

df3.to_excel(path_tela+'gc_india_telagana.xlsx',index=False)

In [ ]:
df4 = pd.read_excel(path_tela+'gc_india_telagana.xlsx')
df4

In [111]:
path = r'C:\Users\stephenl\Downloads\\'
dff = pd.read_excel(path+'sea_account_crm.xlsx',sheet_name='Sheet1')

In [113]:
dff = dff.drop_duplicates('accountname').reset_index(drop=True)

In [114]:
dff.shape

(956, 6)

In [115]:
dff.to_excel(path+'sea_account_crm_final.xlsx',index=False)

In [118]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 956 entries, 0 to 955
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   accountno       956 non-null    object
 1   accountname     956 non-null    object
 2   country         956 non-null    object
 3   class           946 non-null    object
 4   billingaddress  956 non-null    object
 5   salesperson     956 non-null    object
dtypes: object(6)
memory usage: 44.9+ KB


In [88]:
for i in dff.productname:
    if 'Fuji' in i:
        print(i)

Fuji IX GP Extra Capsule
Fuji II LC Capsule
FujiCEM Refill
Fuji VII Capsule Pink
Fuji Plus Capsule
Fuji Ortho LC Intro Pack 1-1
Fuji Coat LC
Fuji Bulk
Fuji Ortho Band Paste Pak
Fuji VII Capsule White
Fuji I Capsule
Fuji Ortho LC Capsule
Fuji VII White


In [ ]:
dff.to_excel('product_master_crm_final.xlsx',index=F)